Some imports

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import csv
import os
import json
from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.regularizers import l1
from tensorflow.keras.optimizers import Adam, Adagrad
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Download the dataset from openML

In [ ]:
dataset = "banknote-authentication"
data = fetch_openml(dataset)
x_data, y_data = data['data'], data['target']

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y_data)
unique = np.unique(y)
y = to_categorical(y, len(unique))

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(x_data, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_val = scaler.fit_transform(X_train_val)
X_test = scaler.transform(X_test)
classes = le.classes_
classes_len = len(classes)

np.save("datasets/"+dataset+'_X_train_val.npy', X_train_val)
np.save("datasets/"+dataset+'_X_test.npy', X_test)
np.save("datasets/"+dataset+'_y_train_val.npy', y_train_val)
np.save("datasets/"+dataset+'_y_test.npy', y_test)
np.save("datasets/"+dataset+'_classes.npy', le.classes_)

with open("datasets/"+dataset+'_sample.csv', 'w') as f:
    write = csv.writer(f)
    write.writerows(X_test)

In [ ]:
train_size = X_train_val.shape[0]
model = Sequential()
f = open('specifics.json')
network_spec = json.load(f)

arch = network_spec["network"]["arch"]
for i in range(0, len(arch)):
    layer_name = network_spec["network"]["arch"][i]["layer_name"]
    activation_function = network_spec["network"]["arch"][i]["activation_function"]
    neurons = network_spec["network"]["arch"][i]["neurons"]
    if i == 0:
        model.add(Dense(neurons, input_shape=(X_train_val.shape[1],), kernel_regularizer=l1(0.0001)))
    else:
        model.add(Dense(neurons, activation=activation_function, name=layer_name, kernel_regularizer=l1(0.0001)))

if  network_spec["network"]["training"]["optimizer"] == "Adam":
    opt = Adam(lr=0.0001)
elif network_spec["network"]["training"]["optimizer"] == "Adagrad":
    opt = Adagrad(lr=0.0001)
else:
    opt = Adam(lr=0.0001)

model.add(Dense(classes_len, activation='softmax'))
model.compile(optimizer=opt, loss=['categorical_crossentropy'], metrics=['accuracy'])

In [ ]:
checkpoint_path = 'models/'+dataset+'/training/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True,verbose=1)
batch_size = int(X_train_val.shape[1]*10) if network_spec["network"]["training"]["batch_size"] == "default" else int(network_spec["network"]["training"]["batch_size"])
epochs = int(network_spec["network"]["training"]["epochs"])
validation_split = 0.25 if network_spec["network"]["training"]["validation_split"] == "default" else float(network_spec["network"]["training"]["validation_split"])
shuffle = True if network_spec["network"]["training"]["shuffle"] == "true" else False

model.fit(
    X_train_val, 
    y_train_val, 
    batch_size=batch_size, 
    epochs=epochs, 
    validation_split=validation_split, 
    shuffle=shuffle,
    callbacks=[cp_callback])

In [ ]:
# DUMP MODEL FOR NEURALBOND

layers = model.layers
weights = model.weights

to_dump = {}

weights = []
nodes = []

# save weigths
for i in range(0 , len(layers)):

    layer_weights = layers[i].get_weights()

    for m in range(0, len(layer_weights)):
        for w in range(0, len(layer_weights[m])):
            try:
                for v in range(0, len(layer_weights[m][w])):
                    weight_info = {
                            "Layer": i+1,
                            "PosCurrLayer": v,
                            "PosPrevLayer": w,
                            "Value": float(layer_weights[m][w][v])
                        }
                    weights.append(weight_info)
            except:
                continue

    
    #print(weights)
    #flat_layer_weigth = [item for sublist in layer_weights for item in sublist]
    if i == 0:
        for units in range(0, layers[i].units):
            weights_l0 = layers[i].get_weights()[0]
            for w in range(0, len(weights_l0)):
                node_info = {
                    "Layer": 0,
                    "Pos": w,
                    "Type": "input",
                    "Bias": 0
                }
                nodes.append(node_info)
            break
    

    for units in range(0, layers[i].units):
        if i == len(layers) - 1:
            bias = layers[i].get_weights()[1]
            node_info = {
                "Layer": i+1,
                "Pos": units,
                "Type": "summation",
                "Bias": bias.tolist()[units]
            }
            nodes.append(node_info)
        else:
            bias = layers[i].get_weights()[1]
            node_info = {
                "Layer": i+1,
                "Pos": units,
                "Type": layers[i].activation.__name__,
                "Bias": bias.tolist()[units]
            }
            nodes.append(node_info)


    if i == len(layers) - 1:
        for l in range(0, len(layer_weights[0][0])):
            node_info = {
                "Layer": i+2,
                "Pos": l,
                "Type": "softmax",
                "Bias": 0
            }
            nodes.append(node_info)


        layer_weights = layers[i-2].get_weights()

        for k in range(0, layers[i].units):
            for l in range(0, layers[i].units):
                weight_info = {
                                "Layer": i+2,
                                "PosCurrLayer": k,
                                "PosPrevLayer": l,
                                "Value": 1
                            }
                weights.append(weight_info)


    if i == len(layers) - 1:
        for l in range(0, len(layer_weights[0][0])):
            node_info = {
                "Layer": i+3,
                "Pos": l,
                "Type": "output",
                "Bias": 0
            }
            nodes.append(node_info)

            weight_info = {
                            "Layer": i+3,
                            "PosCurrLayer": l,
                            "PosPrevLayer": l,
                            "Value": 1
                        }
            weights.append(weight_info)

to_dump["Nodes"] = nodes
to_dump["Weights"] = weights

with open('models/'+dataset+'/modelBM.json', 'w') as fp:
    print(" *** dump model")
    json.dump(to_dump, fp)

In [ ]:
# DUMP KERAS PREDICTION
y_keras = model.predict(X_test)
np.save("datasets/"+dataset+'_y_keras.npy', y_keras)
results = []
for pred in y_keras:
    prediction = np.argmax(pred, axis=0)
    to_save = [pred[0], pred[1] ,prediction]
    results.append(to_save)
    
fields = ['probability_0', 'probability_1', 'classification'] 

with open("datasets/"+dataset+'_swprediction.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(fields)
    write.writerows(results)